# Esper Quick Start Guide

This notebook demonstrates the basic usage of the Esper morphogenetic neural network platform.

## Overview

Esper enables neural networks to autonomously evolve their architecture during training through:
- Dynamic kernel injection at seed points
- GNN-based policy networks for strategic adaptation
- Automated compilation of new computational kernels

Let's start with a simple example!

## 1. Setup and Imports

In [ ]:
# Core imports
import matplotlib.pyplot as plt

# Utilities
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

# Esper imports
from esper import wrap

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 2. Define a Simple Model

We'll start with a basic feedforward neural network for classification.

In [ ]:
class SimpleClassifier(nn.Module):
    """A simple feedforward classifier."""

    def __init__(self, input_dim=784, hidden_dim=256, num_classes=10):
        super().__init__()

        self.features = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.2),
        )

        self.classifier = nn.Linear(hidden_dim // 2, num_classes)

    def forward(self, x):
        features = self.features(x)
        logits = self.classifier(features)
        return logits

# Create model instance
model = SimpleClassifier()
print("Model architecture:")
print(model)

## 3. Enable Morphogenetic Capabilities

Now we'll wrap the model with Esper's morphogenetic capabilities. This allows the model to evolve during training.

In [ ]:
# Wrap the model with morphogenetic capabilities
morphable_model = wrap(
    model,
    target_layers=[nn.Linear],  # Target Linear layers for adaptation
    seeds_per_layer=4,          # Number of adaptation seeds per layer
    device=device
)

# Move to device
morphable_model = morphable_model.to(device)

print("Model wrapped successfully!")
print(f"Number of parameters: {sum(p.numel() for p in morphable_model.parameters()):,}")

## 4. Create Synthetic Dataset

For this demo, we'll create a simple synthetic dataset.

In [ ]:
def create_synthetic_data(num_samples=1000, input_dim=784, num_classes=10):
    """Create a synthetic classification dataset."""
    # Generate random input data
    X = torch.randn(num_samples, input_dim)

    # Generate labels with some structure
    # Create cluster centers for each class
    centers = torch.randn(num_classes, input_dim) * 2

    # Assign each sample to nearest center
    distances = torch.cdist(X, centers)
    y = distances.argmin(dim=1)

    # Add some noise to labels
    noise_mask = torch.rand(num_samples) < 0.1
    y[noise_mask] = torch.randint(0, num_classes, (noise_mask.sum(),))

    return X, y

# Create datasets
X_train, y_train = create_synthetic_data(5000)
X_val, y_val = create_synthetic_data(1000)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

## 5. Training Loop with Adaptation Monitoring

Now we'll train the model and monitor how it adapts over time.

In [ ]:
# Training configuration
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(morphable_model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Tracking metrics
train_losses = []
val_losses = []
val_accuracies = []
adaptation_counts = []

# Training function
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0.0

    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)

# Validation function
def validate(model, loader, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / len(loader)
    accuracy = 100. * correct / total
    return avg_loss, accuracy

In [ ]:
# Training loop
num_epochs = 15

print("Starting training...\n")

for epoch in range(num_epochs):
    # Train
    train_loss = train_epoch(morphable_model, train_loader, criterion, optimizer)
    train_losses.append(train_loss)

    # Validate
    val_loss, val_acc = validate(morphable_model, val_loader, criterion)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    # Update learning rate
    scheduler.step()

    # Get adaptation statistics
    if hasattr(morphable_model, 'get_model_stats'):
        stats = morphable_model.get_model_stats()
        adaptations = stats.get('total_kernel_executions', 0)
        adaptation_counts.append(adaptations)
    else:
        adaptation_counts.append(0)

    # Print progress
    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"  Train Loss: {train_loss:.4f}")
    print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
    print(f"  Total Adaptations: {adaptation_counts[-1]}")
    print()

## 6. Visualize Training Progress

Let's visualize how the model performed and adapted during training.

In [ ]:
# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot training and validation loss
axes[0, 0].plot(train_losses, label='Train Loss', marker='o')
axes[0, 0].plot(val_losses, label='Val Loss', marker='s')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Training Progress')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Plot validation accuracy
axes[0, 1].plot(val_accuracies, color='green', marker='o')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy (%)')
axes[0, 1].set_title('Validation Accuracy')
axes[0, 1].grid(True, alpha=0.3)

# Plot adaptation count
if any(adaptation_counts):
    axes[1, 0].plot(adaptation_counts, color='red', marker='o')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Total Adaptations')
    axes[1, 0].set_title('Morphogenetic Adaptations')
    axes[1, 0].grid(True, alpha=0.3)
else:
    axes[1, 0].text(0.5, 0.5, 'No adaptation data available',
                    ha='center', va='center', transform=axes[1, 0].transAxes)

# Plot adaptation rate
if len(adaptation_counts) > 1:
    adaptation_rate = np.diff(adaptation_counts)
    axes[1, 1].bar(range(1, len(adaptation_counts)), adaptation_rate, color='orange')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('New Adaptations')
    axes[1, 1].set_title('Adaptation Rate per Epoch')
    axes[1, 1].grid(True, alpha=0.3)
else:
    axes[1, 1].text(0.5, 0.5, 'Insufficient data for rate calculation',
                    ha='center', va='center', transform=axes[1, 1].transAxes)

plt.tight_layout()
plt.show()

## 7. Analyze Model Adaptations

Let's examine how the model has adapted its architecture.

In [ ]:
# Get detailed model statistics
if hasattr(morphable_model, 'get_model_stats'):
    stats = morphable_model.get_model_stats()

    print("Model Adaptation Summary:")
    print("=" * 40)
    for key, value in stats.items():
        print(f"{key}: {value}")
    print()

# Get per-layer statistics if available
if hasattr(morphable_model, 'get_layer_stats'):
    layer_stats = morphable_model.get_layer_stats()

    print("\nPer-Layer Adaptation Analysis:")
    print("=" * 40)

    for layer_name, stats in layer_stats.items():
        print(f"\nLayer: {layer_name}")
        for stat_name, stat_value in stats.items():
            print(f"  {stat_name}: {stat_value}")
else:
    print("Detailed adaptation statistics not available.")

## 8. Compare with Non-Morphogenetic Model

Let's train the same architecture without morphogenetic capabilities for comparison.

In [ ]:
# Create and train a regular model
regular_model = SimpleClassifier().to(device)
regular_optimizer = optim.Adam(regular_model.parameters(), lr=0.001)
regular_scheduler = optim.lr_scheduler.StepLR(regular_optimizer, step_size=5, gamma=0.5)

regular_val_accuracies = []

print("Training regular model for comparison...\n")

for epoch in range(num_epochs):
    # Train
    train_loss = train_epoch(regular_model, train_loader, criterion, regular_optimizer)

    # Validate
    val_loss, val_acc = validate(regular_model, val_loader, criterion)
    regular_val_accuracies.append(val_acc)

    # Update learning rate
    regular_scheduler.step()

    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}: Val Acc: {val_acc:.2f}%")

In [ ]:
# Compare performance
plt.figure(figsize=(10, 6))
plt.plot(val_accuracies, label='Morphogenetic Model', marker='o', linewidth=2)
plt.plot(regular_val_accuracies, label='Regular Model', marker='s', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy (%)')
plt.title('Morphogenetic vs Regular Model Performance')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Print final comparison
print("\nFinal Performance:")
print(f"Morphogenetic Model: {val_accuracies[-1]:.2f}%")
print(f"Regular Model: {regular_val_accuracies[-1]:.2f}%")
print(f"Improvement: {val_accuracies[-1] - regular_val_accuracies[-1]:.2f}%")

## 9. Save and Load Models

Esper models can be saved and loaded like regular PyTorch models.

In [ ]:
# Save the morphogenetic model
model_path = "morphogenetic_model.pth"
torch.save({
    'model_state_dict': morphable_model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': num_epochs,
    'val_accuracy': val_accuracies[-1],
}, model_path)

print(f"Model saved to {model_path}")

# Load the model
loaded_model = wrap(
    SimpleClassifier(),
    target_layers=[nn.Linear],
    seeds_per_layer=4,
    device=device
).to(device)

checkpoint = torch.load(model_path)
loaded_model.load_state_dict(checkpoint['model_state_dict'])

print("Model loaded successfully!")
print(f"Loaded from epoch: {checkpoint['epoch']}")
print(f"Validation accuracy: {checkpoint['val_accuracy']:.2f}%")

## 10. Next Steps

Congratulations! You've successfully:
- Created a morphogenetic neural network
- Trained it with autonomous adaptation
- Monitored adaptation statistics
- Compared performance with a regular model

### What's Next?

1. **Try Different Architectures**: Experiment with CNNs, RNNs, or Transformers
2. **Custom Adaptation Policies**: Define your own layer selection strategies
3. **Real Datasets**: Apply to MNIST, CIFAR-10, or your own data
4. **Advanced Configuration**: Use the YAML configs for fine-grained control
5. **Multi-GPU Training**: Scale up with distributed training

### Resources

- [Esper Documentation](../README.md)
- [Example Scripts](../scripts/)
- [Configuration Guide](../configs/)

Happy evolving! 🧬